In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import TimestampType
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

22/08/16 09:22:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/08/16 09:22:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:

# 4
olist_order_payments_dataset = spark.read.options(header='True', inferSchema='True', delimiter=',') \
                            .csv("data/olist_order_payments_dataset.csv")
# 6
olist_orders_dataset = spark.read.options(header='True', inferSchema='True', delimiter=',') \
                            .csv("data/olist_orders_dataset.csv")

In [4]:
payments = olist_order_payments_dataset.select(col("order_id").alias("order_id_2"), "payment_value")
payments.show()

+--------------------+-------------+
|          order_id_2|payment_value|
+--------------------+-------------+
|b81ef226f3fe1789b...|        99.33|
|a9810da82917af2d9...|        24.39|
|25e8ea4e93396b6fa...|        65.71|
|ba78997921bbcdc13...|       107.78|
|42fdf880ba16b47b5...|       128.45|
|298fcdf1f73eb413e...|        96.12|
|771ee386b001f0620...|        81.16|
|3d7239c394a212faa...|        51.84|
|1f78449c87a54faf9...|       341.09|
|0573b5e23cbd79800...|        51.95|
|d88e0d5fa41661ce0...|       188.73|
|2480f727e869fdeb3...|        141.9|
|616105c9352a9668c...|        75.78|
|cf95215a722f3ebf2...|       102.66|
|769214176682788a9...|       105.28|
|12e5cfe0e4716b59a...|       157.45|
|61059985a6fc0ad64...|       132.04|
|79da3f5fe31ad1e45...|        98.94|
|8ac09207f415d55ac...|       244.15|
|b2349a3f20dfbeef6...|       136.71|
+--------------------+-------------+
only showing top 20 rows



In [5]:
olist_orders_dataset.show()

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

In [6]:
join_table = payments.join(olist_orders_dataset, payments.order_id_2==olist_orders_dataset.order_id, "Inner")
join_table = join_table.select("order_id","payment_value","customer_id", "order_purchase_timestamp")
join_table = join_table.withColumn('order_purchase_timestamp',to_date(unix_timestamp \
            (join_table.order_purchase_timestamp, 'yyyy-MM-dd HH:mm:ss').cast('timestamp')))
join_table.show()

+--------------------+-------------+--------------------+------------------------+
|            order_id|payment_value|         customer_id|order_purchase_timestamp|
+--------------------+-------------+--------------------+------------------------+
|e481f51cbdc54678b...|        18.59|9ef432eb625129730...|              2017-10-02|
|e481f51cbdc54678b...|          2.0|9ef432eb625129730...|              2017-10-02|
|e481f51cbdc54678b...|        18.12|9ef432eb625129730...|              2017-10-02|
|53cdb2fc8bc7dce0b...|       141.46|b0830fb4747a6c6d2...|              2018-07-24|
|47770eb9100c2d0c4...|       179.12|41ce2a54c0b03bf34...|              2018-08-08|
|949d5b44dbf5de918...|         72.2|f88197465ea7920ad...|              2017-11-18|
|ad21c59c0840e6cb8...|        28.62|8ab97904e6daea886...|              2018-02-13|
|a4591c265e18cb1dc...|       175.26|503740e9ca751ccdd...|              2017-07-09|
|136cce7faa42fdb2c...|        65.95|ed0271e0b7da060a3...|              2017-04-11|
|651

In [7]:
join_table.createOrReplaceTempView("join_table")

#### 3)	Find maximum spending customers for every month of last year.

What mean last year when the newest data is from 2018?   
Last year of existing data?

In [8]:
spark.sql("""
            SELECT
            max(order_purchase_timestamp)
            FROM join_table
            """).show()

+-----------------------------+
|max(order_purchase_timestamp)|
+-----------------------------+
|                   2018-10-17|
+-----------------------------+



In [9]:
join_table.dtypes

[('order_id', 'string'),
 ('payment_value', 'double'),
 ('customer_id', 'string'),
 ('order_purchase_timestamp', 'date')]

#### Result without filtering for the last year (to be added if necessary)

In [10]:
df = spark.sql("""WITH max_payment AS (
               SELECT
               customer_id,
               EXTRACT(month from `order_purchase_timestamp`) AS month,
               EXTRACT(year from `order_purchase_timestamp`) AS year,
               MAX(payment_value) AS max_cash
               FROM join_table
               GROUP BY customer_id, month, year
               ORDER BY year, month
               )
               SELECT customer_id, month, year, max_cash
               FROM (SELECT 
               *,
               ROW_NUMBER() OVER(PARTITION BY year, month  ORDER BY max_cash DESC) AS row_number
               FROM max_payment)
               WHERE row_number = 1
               
               """)
df.show(50)

+--------------------+-----+----+--------+
|         customer_id|month|year|max_cash|
+--------------------+-----+----+--------+
|08c5351a6aca1c158...|    9|2016|  136.23|
|a9dc96b027d1252bb...|   10|2016| 1423.55|
|0975c3cf2e418ebfa...|   12|2016|   19.62|
|101c8d91278c4de90...|    1|2017| 3016.01|
|c6e2731c5b391845f...|    2|2017| 6929.31|
|c6695e3b1e48680db...|    3|2017| 4016.91|
|df55c14d1476a9a34...|    4|2017| 4950.34|
|3fd6777bbce08a352...|    5|2017| 6726.66|
|4312e816d88fa8bdb...|    6|2017| 3048.27|
|8dd4c93bfbebe2a77...|    7|2017| 3041.73|
|6361b9f3b85d41860...|    8|2017| 2480.58|
|1617b1357756262bf...|    9|2017|13664.08|
|10a86619816f9d2af...|   10|2017|  3126.5|
|05455dfa7cd02f13d...|   11|2017| 6081.54|
|08dc68ad9ab343289...|   12|2017| 2734.66|
|addc91fdf9c2b3045...|    1|2018|  3826.8|
|0b16ce67087d02bf8...|    2|2018| 3358.24|
|35a413c7ca3c69756...|    3|2018| 4175.26|
|e7c905bf4bb13543e...|    4|2018| 3526.46|
|cc803a2c412833101...|    5|2018|  4445.5|
|3d979689f6

Result for 2017

In [22]:
df = spark.sql("""WITH max_payment AS (
               SELECT
               customer_id,
               EXTRACT(month from `order_purchase_timestamp`) AS month,
               EXTRACT(year from `order_purchase_timestamp`) AS year,
               MAX(payment_value) AS max_cash
               FROM join_table
               GROUP BY customer_id, month, year
               ORDER BY year, month
               )
               SELECT customer_id, month, year, max_cash
               FROM (SELECT 
               *,
               ROW_NUMBER() OVER(PARTITION BY year, month  ORDER BY max_cash DESC) AS row_number
               FROM max_payment)
               WHERE row_number = 1 AND year = '2017'
               
               """)
df.show(50)

+--------------------+-----+----+--------+
|         customer_id|month|year|max_cash|
+--------------------+-----+----+--------+
|101c8d91278c4de90...|    1|2017| 3016.01|
|c6e2731c5b391845f...|    2|2017| 6929.31|
|c6695e3b1e48680db...|    3|2017| 4016.91|
|df55c14d1476a9a34...|    4|2017| 4950.34|
|3fd6777bbce08a352...|    5|2017| 6726.66|
|4312e816d88fa8bdb...|    6|2017| 3048.27|
|8dd4c93bfbebe2a77...|    7|2017| 3041.73|
|6361b9f3b85d41860...|    8|2017| 2480.58|
|1617b1357756262bf...|    9|2017|13664.08|
|10a86619816f9d2af...|   10|2017|  3126.5|
|05455dfa7cd02f13d...|   11|2017| 6081.54|
|08dc68ad9ab343289...|   12|2017| 2734.66|
+--------------------+-----+----+--------+



In [ ]:
a differenty customer spend maximum amount every month

In [24]:
df.select('customer_id').distinct().count()

12

In [26]:
type(df)

pyspark.sql.dataframe.DataFrame

In [27]:
df = df.toPandas()

In [28]:
import plotly.express as px

In [34]:

fig = px.bar(df, x='month', y='max_cash', color='customer_id', color_discrete_sequence=px.colors.qualitative.Dark24,
             title='Maximum spending customers fo every month',
             labels={
                 "max_cash":"Amount spent",
                 "customer_id": "Customer ID",
                 "month": 'Month'
             })
fig.show()